## 모델 성능 평가 및 개선

### 내용
1.  모델 성능 평가 지표
    - 평가 지표의 중요성
        - 모델 성능을 객관적으로 평가하기 위한 지표 필요성
        - 다양한 지표를 통해 모델의 강점과 약점 파악
    - 분류 문제의 평가 지표
        - 정확도(Accuracy)
        - 정밀도(Precision)
        - 재현율(Recall)
        - F1 스코어(F1 Score)
        - ROC-AUC (Receiver Operating Characteristic - Area Under Curve)
 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 예측값 생성
y_pred = model.predict(X_test) # 모델 없음

# 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC-AUC: {roc_auc}')

- 회귀 문제의 평가 지표
    - 평균 절대 오차(MAE, Mean Absolute Error)
    - 평균 제곱 오차(MSE, Mean Squared Error)
    - 결정 계수(R² Score)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 예측값 생성
y_pred = model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

2. 모델 성능 개선 방법
    - 데이터 증강(Data Augmentation)
        - 데이터 다양성 증가를 통한 일반화 성능 향상

In [2]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

- 하이퍼파라미터 튜닝(Hyperparameter Tuning)
    - 최적의 하이퍼파라미터 조합 찾기

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}

model = RandomForestClassifier()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

print(f'Best parameters found: {grid_search.best_params_}')


- 앙상블 기법(Ensemble Methods)
    - 여러 모델의 예측을 결합하여 성능 향상

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

clf1 = LogisticRegression()
clf2 = SVC(probability=True)
clf3 = RandomForestClassifier()

ensemble_model = VotingClassifier(estimators=[
    ('lr', clf1), ('svc', clf2), ('rf', clf3)], voting='soft')

ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble model accuracy: {accuracy}')

3. 프로젝트 설명 및 진행
    - 프로젝트 개요
        - 모델 성능 평가 및 개선을 위한 실전 프로젝트 수행
        - 평가 지표를 사용하여 모델 성능을 측정하고, 다양한 기법을 통해 성능 개선 시도
    - 데이터셋 선택
        - Kaggle 또는 기타 오픈 데이터셋에서 프로젝트 데이터셋 선택
        - CIFAR-10, Dogs vs. Cats, Titanic 등
    - 프로젝트 단계
        - 데이터 전처리 및 준비
        - 모델 설계 및 학습
        - 성능 평가 지표 적용
        - 모델 성능 개선 기법 적용
        - 성능 평가 및 결과 분석

4. 프로젝트 예시
    - 예시 프로젝트: CIFAR-10 이미지 분류
        - 데이터셋 로드 및 전처리
        - 성능 평가 지표 적용 및 모델 성능 개선 기법 적용

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

# 데이터 전처리 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Pretrained ResNet 모델 로드 및 수정
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 학습 루프
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()  # 학습률 갱신
    val_loss = evaluate_model(model, val_loader)  # 검증 손실 계산

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Learning Rate: {scheduler.get_last_lr()}')

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

5. 프로젝트 발표 및 코드 리뷰
    - 프로젝트 발표
        - 각 그룹 또는 개인은 프로젝트 결과 발표
        - 모델 설계, 훈련 과정, 성능 평가 결과 등을 공유
    - 코드 리뷰 및 피드백
        - 각 그룹의 코드를 리뷰하고 피드백 제공
        - 개선할 점 및 좋은 점 공유

6. 실습
    - 다양한 성능 평가 지표를 적용하여 모델 성능 분석
    - 모델 성능 개선 기법을 적용하여 결과 분석

In [ ]:
# 실습 과제 예시
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 모델 학습 및 평가
model = RandomForestClassifier(n_estimators=100, max_depth=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 성능 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')